In [8]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [9]:
# import torch
# from transformers import AutoModelForCausalLM
# from transformers import AutoTokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from scipy.special import softmax
import numpy as np
import torch
from datasets import load_dataset



In [29]:
# text = "APA president Philip G. Zimbardo of Stanford says they have not had much opposition against this." # 2.0|||1.6666666666666667|||27.0
# text1 = "Except for this small vocal minority, we have not gotten a lot of groundswell against this, says APA president Zimbardo." # 3.0|||3.0|||6.0

text = "Mr. Ackerman said the buy-back and above - market price, prove Mr. Edelman is scared." #|||3.0|||3.0|||6.0
text1 = "The buy - back, and the above - price prove Mr. Edelman is scared, Mr. Ackerman says." # 1.6666666666666667|||2.0|||26.0

In [20]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [49]:
def tok(text):
    
    input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)  # Batch size 1
    tokenize_input = tokenizer.tokenize(text)
    #50256 is the token_id for <|endoftext|>
    tensor_input = torch.tensor([ [50256]  +  tokenizer.convert_tokens_to_ids(tokenize_input)])
    with torch.no_grad():
        outputs = model(tensor_input, labels=tensor_input)
        loss, logits = outputs[:2]
    print("a=", loss*len(tokenize_input))

    lp = 0.0
    for i in range(len(tokenize_input)):
        masked_index = i
        predicted_score = logits[0, masked_index]
        predicted_prob = softmax(np.array(predicted_score))
        lp += np.log(predicted_prob[tokenizer.convert_tokens_to_ids([tokenize_input[i]])[0]])

    return lp

lp = tok(text)
lp1 = tok(text1)

print("lp=", lp)
print("lp1=", lp1)

a= tensor(96.0704)
a= tensor(117.6237)
lp= -96.07070533093065
lp1= -117.62397847138345


In [19]:
count={}
Total_count=0
dataset = load_dataset("gigaword")
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 3803957
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 189651
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 1951
    })
})


In [22]:

for line in dataset['train']['document']:
    line = tokenizer.tokenize(line)
    for word in line:
        if word in count:
            count[word] += 1
        else:
            count[word] = 1
        Total_count += 1

In [40]:
def calc_unigram(text):
    tokenize_input = tokenizer.tokenize(text)
    lp = 0.0
    for i in range(len(tokenize_input)):
        if(tokenize_input[i] not in count):
            lp += np.log(1/Total_count)
        else:
            lp += np.log(count[tokenize_input[i]]/Total_count)
    return lp
def slor(text):
    return (tok(text) - calc_unigram(text))/(len(tokenizer.tokenize(text))+1)

In [50]:
x = slor(text)
x1 = slor(text1)   
print("slor=", x)
print("slor1=", x1)

a= tensor(96.0704)
a= tensor(117.6237)
slor= 5.69299403834961
slor1= 5.189822636127331


In [47]:
slor_1 =[]

with open('./RawData/temp.tsv','r') as f:
    for lines in f:
        x=lines.split('|||')
        score = slor(x[1])
        slor_1.append(score)

slor_1 = np.array(slor_1)

print(slor_1.shape)
print(slor_1[0:10])

a= tensor(97.8270)
a= tensor(99.9071)
a= tensor(75.4009)
a= tensor(75.1400)
a= tensor(75.3536)
a= tensor(85.9226)
a= tensor(82.0556)
a= tensor(122.1084)
a= tensor(101.7406)
a= tensor(92.5261)
a= tensor(74.7845)
a= tensor(93.7213)
a= tensor(84.3001)
a= tensor(114.8254)
a= tensor(117.7278)
a= tensor(82.1570)
a= tensor(120.5903)
a= tensor(108.6104)
a= tensor(118.8474)
a= tensor(121.4158)
a= tensor(118.1443)
a= tensor(84.3896)
a= tensor(81.3573)
a= tensor(75.8898)
a= tensor(80.0790)
a= tensor(82.5900)
a= tensor(85.8004)
a= tensor(73.9457)
a= tensor(114.9744)
a= tensor(95.0970)
a= tensor(81.9452)
a= tensor(83.0099)
a= tensor(85.2521)
a= tensor(64.0629)
a= tensor(70.7485)
a= tensor(105.3573)
a= tensor(93.3861)
a= tensor(83.1568)
a= tensor(92.4871)
a= tensor(93.1113)
a= tensor(128.1174)
a= tensor(83.5485)
a= tensor(103.5113)
a= tensor(97.2595)
a= tensor(79.0088)
a= tensor(75.8478)
a= tensor(70.4336)
a= tensor(182.7107)
a= tensor(80.6327)
a= tensor(163.6070)
a= tensor(108.8568)
a= tensor(115.3

KeyboardInterrupt: 

In [1]:
def NCE(text):
    return tok(text)/len(tokenizer.tokenize(text)+1)
def PPL(text):
    return np.exp(-NCE(text))